# Introdução


In [0]:
!git clone https://github.com/henriquepgomide/caRtola

Cloning into 'caRtola'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 2083 (delta 77), reused 52 (delta 22), pack-reused 1968
Receiving objects: 100% (2083/2083), 178.43 MiB | 27.91 MiB/s, done.
Resolving deltas: 100% (1154/1154), done.


In [0]:
import os
rootdir = os.curdir
os.chdir('caRtola/data')
os.listdir()

['2018',
 '2016',
 '2014',
 'dados_agregados_amostras.csv',
 'dados_agregados_limpos.csv',
 'posicoes_ids.csv',
 '2017',
 'README.md',
 '2015',
 'times_ids.csv',
 'dados_agregados.csv']

# Objetivo
```
# This is formatted as code
```



# Metodologia


# Análise


## Limpeza

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import _pickle as plk
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

%matplotlib inline

pd.set_option('display.max_columns', 100)

In [118]:
df = pd.read_csv('dados_agregados.csv')
print(df.shape)
df.head(10)

(43821, 77)


,AtletaID,Rodada,ClubeID,Participou,Posicao,Jogos,Pontos,PontosMedia,Preco,PrecoVariacao,FS,PE,A,FT,FD,FF,G,I,PP,RB,FC,GC,CA,CV,SG,DD,DP,GS,ano,Apelido,Status,avg.Points,avg.last05,avg.FS,avg.FS.l05,avg.PE,avg.PE.l05,avg.A,avg.A.l05,avg.FT,avg.FT.l05,avg.FD,avg.FD.l05,avg.FF,avg.FF.l05,avg.G,avg.G.l05,avg.I,avg.I.l05,avg.PP,avg.PP.l05,avg.RB,avg.RB.l05,avg.FC,avg.FC.l05,avg.GC,avg.GC.l05,avg.CA,avg.CV.l05,avg.SG,avg.SG.l05,avg.DD,avg.DD.l05,avg.DP,avg.DP.l05,avg.GS,avg.GS.l05,risk_points,mes,dia,home.score.x,away.score.x,pred.home.score,pred.away.score,home.attack,home.defend,variable
0,36443,1,285.0,True,gol,1.0,5.0,5.00,10.60,1.60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2014,NaN,NaN,2.500000,1.00,0.000000,1.0,0.000000,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.000000,1.0,0.0,1.0,0.000000,1.0,0.500000,1.0,0.000000,1.0,0.0,1.0,0.000000,1.0,1.000000,4.0,19.0,1.0,0.0,NaN,NaN,NaN,NaN,home.team
1,36443,2,285.0,True,gol,2.0,-3.0,1.00,8.27,-2.33,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2014,NaN,NaN,0.666667,1.00,0.666667,1.0,0.000000,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.000000,1.0,0.0,1.0,0.000000,1.0,0.333333,1.0,0.000000,1.0,0.0,1.0,0.666667,1.0,1.000000,4.0,27.0,2.0,2.0,NaN,NaN,NaN,NaN,away.team
2,36443,3,285.0,True,gol,3.0,-2.6,-0.20,6.81,-1.46,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2014,NaN,NaN,-0.150000,1.00,0.500000,1.0,0.500000,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.000000,1.0,0.0,1.0,0.000000,1.0,0.250000,1.0,0.000000,1.0,0.0,1.0,0.750000,1.0,1.000000,5.0,4.0,2.0,1.0,NaN,NaN,NaN,NaN,home.team
3,36443,4,285.0,True,gol,4.0,4.0,0.85,7.96,1.15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,2014,NaN,NaN,0.680000,0.68,0.400000,0.4,0.400000,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.200000,0.2,0.400000,0.4,0.0,0.0,0.800000,0.8,1.000000,5.0,10.0,2.0,1.0,NaN,NaN,NaN,NaN,home.team
4,36443,5,285.0,True,gol,5.0,5.0,1.68,8.61,0.65,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2014,NaN,NaN,1.400000,1.68,0.333333,0.4,0.333333,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.333333,0.4,0.333333,0.4,0.0,0.0,0.666667,0.8,1.000000,5.0,18.0,0.0,0.0,NaN,NaN,NaN,NaN,away.team
5,36443,6,285.0,True,gol,6.0,1.2,1.60,8.02,-0.59,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,2,0,1,2014,NaN,NaN,1.371429,0.92,0.285714,0.4,0.428571,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.2,0.0,0.0,0.142857,0.0,0.285714,0.2,0.571429,0.8,0.0,0.0,0.714286,1.0,1.000000,5.0,21.0,1.0,1.0,NaN,NaN,NaN,NaN,away.team
6,36443,7,285.0,True,gol,7.0,-3.3,0.90,6.75,-1.27,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3,2014,NaN,NaN,0.787500,0.86,0.250000,0.0,0.500000,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125000,0.2,0.0,0.0,0.125000,0.0,0.250000,0.2,0.625000,1.0,0.0,0.0,1.000000,1.2,1.000000,5.0,25.0,1.0,3.0,NaN,NaN,NaN,NaN,home.team
7,36443,8,285.0,True,gol,8.0,3.0,1.16,7.55,0.80,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,2014,NaN,NaN,1.033333,1.98,0.222222,0.0,0.444444,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.2,0.0,0.0,0.222222,0.0,0.333333,0.4,0.555556,1.0,0.0,0.0,0.888889,1.0,1.000000,5.0,28.0,2.0,0.0,NaN,NaN,NaN,NaN,home.team
8,36443,9,285.0,True,gol,9.0,-1.5,0.86,6.71,-0.84,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2014,NaN,NaN,0.780000,0.88,0.300000,0.2,0.400000,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.2,0.0,0.0,0.200000,0.0,0.300000,0.4,0.500000,0.6,0.0,0.0,0.900000,1.0,3.320910,6.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,away.team
9,36443,10,285.0,True,gol,10.0,-4.0,0.37,6.01,-0.70,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2014,NaN,NaN,0.345455,-0.92,0.272727,0.2,0.363636,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090909,0.2,0.0,0.0,0.181818,0.0,0.272727,0.2,0.454545,0.6,0.0,0.0,1.000000,1.4,3.649901,7.0,17.0,2.0,1.0,NaN,NaN,NaN,NaN,away.team


In [0]:
df_1 = pd.read_csv('2018/rodada-1.csv', index_col=0)

df_1[df_1['G'] == df_1['G'].max()]

,atletas.nome,atletas.slug,atletas.apelido,atletas.foto,atletas.atleta_id,atletas.rodada_id,atletas.clube_id,atletas.posicao_id,atletas.status_id,atletas.pontos_num,atletas.preco_num,atletas.variacao_num,atletas.media_num,atletas.clube.id.full.name,FC,FD,FF,FS,G,I,RB,CA,PE,A,SG,DD,FT,GS,CV,GC
5,Rodrigo Eduardo Costa Marinho,rodriguinho,Rodriguinho,https://s.glbimg.com/es/sde/f/2018/03/20/c125f...,61033,1,COR,mei,Provável,16.5,23.08,6.08,16.5,Corinthians,1.0,NaN,NaN,2.0,2.0,NaN,1.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,Sergio Antonio Soler de Oliveira Junior,serginho,Serginho,https://s.glbimg.com/es/sde/f/2018/03/17/144f9...,88002,1,AME,mei,Provável,17.9,15.78,10.78,17.9,América-MG,1.0,NaN,1.0,NaN,2.0,1.0,2.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
494,Nicolás Federico López Alonso,nico-lopez,Nico López,https://s.glbimg.com/es/sde/f/2018/04/08/5bb6e...,84709,1,INT,ata,Provável,18.8,17.77,10.77,18.8,Internacional,NaN,1.0,2.0,1.0,2.0,1.0,1.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [203]:
#TODO: improve code.
import re

year='2018'

df_dict = {}
for f in os.listdir(year):
  match = re.match(r'rodada-\d+.csv', f)
  if match:
    r = match.group(0).split('-')[1].split('.')[0]
    df_dict[int(r)] = pd.read_csv('2018/' + f, index_col=0)

df_dict = {key:df_dict[key] for key in sorted(df_dict.keys())}
  
df_2018 = pd.concat(df_dict.values())

,A,CA,CV,DD,DP,FC,FD,FF,FS,FT,G,GC,GS,I,PE,PP,RB,SG,atletas.apelido,atletas.atleta_id,atletas.clube.id.full.name,atletas.clube_id,atletas.foto,atletas.media_num,atletas.nome,atletas.pontos_num,atletas.posicao_id,atletas.preco_num,atletas.rodada_id,atletas.slug,atletas.status_id,atletas.variacao_num
1,5.0,NaN,NaN,NaN,NaN,10.0,5.0,8.0,7.0,1.0,3.0,NaN,NaN,NaN,53.0,NaN,4.0,NaN,Jadson,38315,Corinthians,COR,https://s.glbimg.com/es/sde/f/2018/03/20/6abc8...,3.12,Jadson Rodrigues da Silva,0.00,mei,7.16,27,jadson,Provável,0.00
2,NaN,NaN,NaN,29.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,18.0,NaN,8.0,NaN,NaN,7.0,Fábio,37656,Cruzeiro,CRU,https://s.glbimg.com/es/sde/f/2018/05/18/d4072...,4.14,Fábio Deivson Lopes Maciel,0.00,gol,12.23,27,fabio,Nulo,0.00
3,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,7.0,NaN,3.0,1.0,Juan,36540,Flamengo,FLA,https://s.glbimg.com/es/sde/f/2018/05/04/a27fb...,1.55,Juan Silveira dos Santos,0.00,zag,5.74,27,juan,Contundido,0.00
4,NaN,4.0,NaN,NaN,NaN,10.0,NaN,NaN,6.0,NaN,1.0,NaN,NaN,NaN,19.0,NaN,4.0,4.0,Paulo André,37604,Atlético-PR,ATL,https://s.glbimg.com/es/sde/f/2018/03/01/b7709...,2.29,Paulo André Cren Benini,-4.00,zag,5.67,27,paulo-andre,Provável,-0.44
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Guto Ferreira,37245,Chapecoense,CHA,https://s.glbimg.com/es/sde/f/2018/08/07/272c6...,3.21,Augusto Sérgio Ferreira,0.93,tec,8.22,27,guto-ferreira,Provável,0.08


In [0]:
matches = pd.read_csv('2018/2018_partidas.csv', index_col=0)

def split_score_home(score):
  if len(score) > 1:
    s = score.split(' x ')
    return int(s[0])
  else:
    return np.nan
  
def split_score_away(score):
  if len(score) > 1:
    s = score.split(' x ')
    return int(s[1])
  else:
    return np.nan
  
 
matches['home_score'] = matches['score'].apply(split_score_home).values
matches['away_score'] = matches['score'].apply(split_score_away).values

# BUG: Column X wtf.
matches.drop('X', axis=1, inplace=True)

In [223]:
#Melhores goleiros
df_goleiros = df_2018[df_2018['atletas.posicao_id'] == 'gol']
df_goleiros = df_goleiros[['atletas.rodada_id', 'atletas.atleta_id', 'atletas.club_id', 'atletas.nome', 'at']]


,A,CA,CV,DD,DP,FC,FD,FF,FS,FT,G,GC,GS,I,PE,PP,RB,SG,atletas.apelido,atletas.atleta_id,atletas.clube.id.full.name,atletas.clube_id,atletas.foto,atletas.media_num,atletas.nome,atletas.pontos_num,atletas.posicao_id,atletas.preco_num,atletas.rodada_id,atletas.slug,atletas.status_id,atletas.variacao_num
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fernando Leal,38350,América-MG,AME,https://s.glbimg.com/es/sde/f/2018/03/20/60207...,0.0,Fernando Leal Fonseca,0.0,gol,3.00,1,fernando-leal,Nulo,0.00
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Glauco,84555,América-MG,AME,https://s.glbimg.com/es/sde/f/2018/03/17/e3246...,0.0,Glauco Tadeu Chaves,0.0,gol,4.00,1,glauco,Nulo,0.00
21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,João Ricardo,81881,América-MG,AME,https://s.glbimg.com/es/sde/f/2018/03/17/8f235...,0.0,João Ricardo Riedi,0.0,gol,6.00,1,joao-ricardo,Contundido,0.00
31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Erick,97930,América-MG,AME,https://s.glbimg.com/es/sde/f/2018/03/17/163ac...,0.0,Erick Martins Ferreira Carvalho,0.0,gol,1.00,1,erick,Nulo,0.00
34,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,Jory,97453,América-MG,AME,https://s.glbimg.com/es/sde/f/2018/03/17/3e918...,13.7,Joriwinnyson Santos dos Anjos Rodrigues,13.7,gol,10.13,1,jory,Provável,9.13


,game,round,date,home_team,score,away_team,arena,home_score,away_score
1,1,1,14/04/2018 - 16:00,Cruzeiro - MG,0 x 1,Grêmio - RS,Mineirão - Belo Horizonte - MG,0.0,1.0
2,2,1,15/04/2018 - 19:00,Atlético - PR,5 x 1,Chapecoense - SC,Arena da Baixada - Curitiba - PR,5.0,1.0
3,3,1,15/04/2018 - 11:00,América - MG,3 x 0,Sport - PE,Independência - Belo Horizonte - MG,3.0,0.0
4,4,1,14/04/2018 - 19:00,Vitória - BA,2 x 2,Flamengo - RJ,Manoel Barradas - Salvador - BA,2.0,2.0
5,5,1,15/04/2018 - 16:00,Vasco da Gama - RJ,2 x 1,Atlético - MG,São Januário - Rio de Janeiro - RJ,2.0,1.0
